In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
from sklearn.model_selection import train_test_split

from zipfile import ZipFile
import os
import sys

# Load modules from the lib directory

#sys.path.insert(0, "../lib")
#from resnet import *

# BUILDING A MODEL

In [2]:
def build_image_classifier(cnn: tf.keras.Model) -> tf.keras.Model:
    """
    Build a new model that takes images as input and outputs class probabilities.

    Parameters:
    -----------

    `cnn: tf.keras.Model` 
        The base CNN model to use.

    `return`
        The new model.
    """
    return tf.keras.Sequential([
        cnn,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units = 512),
        tf.keras.layers.Dense(units = 10, activation = "sigmoid")
    ])

In [3]:
class LabelCleaner(tf.keras.Model):

    def __init__(self, CNN: tf.keras.Model):
        super(LabelCleaner, self).__init__()

        # Base CNN model
        self.CNN = CNN
        
        # Fully connected dense layers
        self.fc_1 = tf.keras.layers.Dense(units = 20, use_bias=False)
        self.fc_2 = tf.keras.layers.Dense(units = 512)
        self.fc_3 = tf.keras.layers.Dense(units = 512, use_bias=False, activation = "relu")
        self.fc_4 = tf.keras.layers.Dense(units = 10, use_bias=False,)
        
        # Batch Normalization layers
        self.bn_1 = tf.keras.layers.BatchNormalization()
        self.bn_2 = tf.keras.layers.BatchNormalization()
        self.bn_3 = tf.keras.layers.BatchNormalization()

    def call(self, inputs):
        img, y = inputs

        # Get the CNN output
        x = self.CNN(img)

        # Embed the output of the CNN to the noisy labels
        x = tf.concat([x, y], axis = 1)
        
        x = self.fc_1(x)    # Linear followed by batch normalization
        x = self.bn_1(x)

        x = self.fc_2(x)    # Linear followed by batch normalization
        x = self.bn_2(x)

        x = self.fc_3(x)    # ReLU

        x = self.fc_4(x)    # Linear followed by batch normalization
        x = self.bn_3(x) #lets see

        x = x + y           # Residual connection

        return x

In [16]:
resnet50 = tf.keras.applications.ResNet50(weights="imagenet", include_top=False, input_shape=(32, 32, 3))
resnet50.trainable = False
cnn_2 = tf.keras.models.Sequential([
    resnet50,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units = 10, activation = "softmax")
])

# MAKE A DATASET

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
if not os.path.exists("./drive"):
    drive.mount('/content/drive')

if not os.path.exists("../data"):
    os.mkdir("../data")

# loading the temp.zip and creating a zip object
with ZipFile("./drive/MyDrive/train_data.zip", 'r') as zip_object:

    # Extracting all the members of the zip 
    # into a specific location.
    zip_object.extractall(path="../data")


In [6]:
# [DO NOT MODIFY THIS CELL]

n_images: int = 50_000
n_noisy: int = 40_000
n_clean: int = n_images - n_noisy

images : np.ndarray = np.empty((n_images, 32, 32, 3), dtype=np.float32)

# Load the data
for i in range(n_images):
    image_path = f"../data/images/{i+1:05d}.png"
    images[i,:,:,:] = cv2.cvtColor(cv2.imread(image_path),cv2.COLOR_BGR2RGB)

# load the labels
clean_labels = np.genfromtxt('../data/clean_labels.csv', delimiter=',', dtype="int8")
noisy_labels = np.genfromtxt('../data/noisy_labels.csv', delimiter=',', dtype="int8")

In [7]:
test_ratio: float = 0.2
train_size: float = n_images - (n_clean * test_ratio)
clean_noisy_ratio: float = 1 / 9
train_clean_size: int = int(np.floor(train_size * clean_noisy_ratio))
val_clean_size: int = int(np.floor((n_clean * (1 - test_ratio)) - train_clean_size))
test_clean_size: int = n_clean - train_clean_size - val_clean_size

IMG_SIZE: int = 32
IMG_SHAPE: tuple = (IMG_SIZE, IMG_SIZE, 3)

BATCH_SIZE: int = 32

In [8]:
images_normalized = tf.cast(images, dtype = tf.float32) / 255.0
clean_labels_one_hot = tf.one_hot(clean_labels, depth = 10)
noisy_labels_one_hot = tf.one_hot(noisy_labels, depth = 10)

In [9]:
T = tf.data.Dataset.from_tensor_slices((images_normalized, noisy_labels_one_hot))
V = tf.data.Dataset.from_tensor_slices((
    (images_normalized[:n_clean],
    noisy_labels_one_hot[:n_clean]),
    clean_labels_one_hot[:n_clean]
))

In [10]:
T = T.shuffle(buffer_size = 1000)
T = T.batch(BATCH_SIZE)

In [11]:

V = V.shuffle(buffer_size = 1000, seed = 42)
V_train = V.take(train_clean_size).batch(batch_size = BATCH_SIZE)
V_val = V.skip(train_clean_size).take(val_clean_size).batch(batch_size = BATCH_SIZE)
V_test = V.skip(train_clean_size + val_clean_size).take(test_clean_size).batch(batch_size = BATCH_SIZE)

In [12]:
# Kind of cleaned dataset
cl1 = tf.data.Dataset.from_tensor_slices((
    (images_normalized[1000:10000],
    noisy_labels_one_hot[1000:10000]),
    clean_labels_one_hot[1000:10000]
))

#cl1=cl1.shuffle(buffer_size=1000)
cl1=cl1.shuffle(buffer_size=9000)
cl1_train= cl1.take(8100).batch(batch_size = BATCH_SIZE)
#.batch(batch_size = BATCH_SIZE)
cl1_val= cl1.skip(8100).batch(batch_size = BATCH_SIZE)
#.batch(batch_size = BATCH_SIZE)
cl1_test= tf.data.Dataset.from_tensor_slices((
    (images_normalized[:1000],
    noisy_labels_one_hot[:1000]),
    clean_labels_one_hot[:1000]
)).batch(batch_size = BATCH_SIZE)
#.batch(batch_size = BATCH_SIZE)

#cl1 = cl1.shuffle(buffer_size = 1000, seed = 42)
#cl1_train= cl1.take(8100).batch(batch_size = BATCH_SIZE)
#cl1_val = cl1.skip(train_clean_size).take(val_clean_size).batch(batch_size = BATCH_SIZE)

#cl1_train = V.take(train_clean_size).batch(batch_size = BATCH_SIZE)
#V_val = V.skip(train_clean_size).take(val_clean_size).batch(batch_size = BATCH_SIZE)
#V_test = V.skip(train_clean_size + val_clean_size).take(test_clean_size).batch(batch_size = BATCH_SIZE)

In [13]:
len(list(cl1))

9000

In [14]:
@tf.function
def l1_loss(y_true, y_pred):
    return tf.reduce_sum(tf.abs(y_true - y_pred))

In [17]:
cleaner = LabelCleaner(cnn_2)
cleaner.compile(
    optimizer = tf.keras.optimizers.Adam(0.001),
    loss = l1_loss,
    metrics = ['accuracy']
)

In [18]:
%%time
# cleaner.fit(
#     V_train,
#     epochs = 30,
#     validation_data = V_val
# )

early_stopping_cl = tf.keras.callbacks.EarlyStopping(patience=10)
cleaner.fit(cl1_train,epochs = 30,validation_data = cl1_val,callbacks=[early_stopping_cl])
    #images[1000:10000], clean_labels[1000:10000] ,#clean_labels[1000:10000],
           # validation_split = 0.2,epochs = 30)
    #validation_data = V_val


Epoch 1/30
254/254 [==============================] - 46s 162ms/step - loss: 158.9236 - accuracy: 0.1168 - val_loss: 72.4253 - val_accuracy: 0.2778
Epoch 2/30
254/254 [==============================] - 42s 167ms/step - loss: 104.2556 - accuracy: 0.1373 - val_loss: 82.4599 - val_accuracy: 0.1867
Epoch 3/30
254/254 [==============================] - 43s 168ms/step - loss: 72.5850 - accuracy: 0.1400 - val_loss: 50.4256 - val_accuracy: 0.1111
Epoch 4/30
254/254 [==============================] - 44s 172ms/step - loss: 47.5321 - accuracy: 0.2395 - val_loss: 36.8495 - val_accuracy: 0.1944
Epoch 5/30
254/254 [==============================] - 42s 167ms/step - loss: 41.6171 - accuracy: 0.3520 - val_loss: 36.7041 - val_accuracy: 0.4267
Epoch 6/30
254/254 [==============================] - 43s 170ms/step - loss: 40.1767 - accuracy: 0.3846 - val_loss: 36.5409 - val_accuracy: 0.3811
Epoch 7/30
254/254 [==============================] - 47s 185ms/step - loss: 39.9899 - accuracy: 0.3902 - val_loss: 

In [19]:
cleaner.evaluate(V_test)

63/63 [==============================] - 11s 170ms/step - loss: 37.9426 - accuracy: 0.3895


[37.942623138427734, 0.3894999921321869]

In [20]:
cleaned_labels_labclean = cleaner.predict([images, noisy_labels_one_hot], batch_size=128)

391/391 [==============================] - 177s 448ms/step


In [22]:
import sklearn
sklearn.metrics.accuracy_score(clean_labels[:1000],np.argmax(cleaned_labels_labclean , axis=1)[:1000],)


0.38

In [27]:
import pickle
# Save the trained model as a pickle string.
saved_model_labclean = pickle.dumps(cleaner)
lab_clean_from_pickle = pickle.loads(saved_model_labclean )
  
# Use the loaded pickled model to make predictions
lab_clean_from_pickle.predict(images_normalized[:1000])

ValueError: ignored

# TRAINING IMAGE CLASSIFIER

In [23]:
from keras.applications.vgg16 import VGG16
vgg = VGG16(input_shape=(32,32,3), include_top=False,weights='imagenet')
vgg.trainable = False

In [28]:
image_classifier2 = build_image_classifier(vgg)

In [29]:
image_classifier2.compile(
    loss = tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy']
)

In [ ]:
%%time
image_classifier_model=image_classifier2.fit(images_normalized[1000:],tf.one_hot(np.argmax(cleaned_labels_labclean, axis=1),depth=10 )[1000:],epochs=10,validation_split=0.1)



Epoch 1/10
1379/1379 [==============================] - 492s 357ms/step - loss: 2.2166 - accuracy: 0.2229 - val_loss: 2.2531 - val_accuracy: 0.2120
Epoch 2/10
1379/1379 [==============================] - 495s 359ms/step - loss: 2.2136 - accuracy: 0.2248 - val_loss: 2.2488 - val_accuracy: 0.2000
Epoch 3/10
1379/1379 [==============================] - 491s 356ms/step - loss: 2.2068 - accuracy: 0.2317 - val_loss: 2.2340 - val_accuracy: 0.2129
Epoch 4/10
1379/1379 [==============================] - 498s 361ms/step - loss: 2.2029 - accuracy: 0.2358 - val_loss: 2.2350 - val_accuracy: 0.2096
Epoch 5/10
1379/1379 [==============================] - 491s 356ms/step - loss: 2.1999 - accuracy: 0.2309 - val_loss: 2.2394 - val_accuracy: 0.2235
Epoch 6/10
1379/1379 [==============================] - 482s 350ms/step - loss: 2.1967 - accuracy: 0.2364 - val_loss: 2.2381 - val_accuracy: 0.2147
Epoch 7/10
1379/1379 [==============================] - 481s 349ms/step - loss: 2.1942 - accuracy: 0.2383 - val_

In [66]:
vgg_pred=image_classifier2.predict(images_normalized[:1000])

32/32 [==============================] - 10s 316ms/step


In [75]:
import sklearn
sklearn.metrics.accuracy_score(clean_labels[:1000],np.argmax(vgg_pred,axis=1))

0.358

In [97]:
import pickle
# Save the trained model as a pickle string.
saved_model_VGG = pickle.dumps(vgg)

In [102]:
vgg_from_pickle = pickle.loads(saved_model_VGG)
  
# Use the loaded pickled model to make predictions
vgg_from_pickle.predict(images_normalized[:1000])

32/32 [==============================] - 10s 314ms/step


array([[[[0.1017667 , 0.        , 0.84405017, ..., 0.        ,
          0.39529675, 0.        ]]],


       [[[0.9722156 , 0.        , 0.58577245, ..., 0.        ,
          0.7981682 , 0.        ]]],


       [[[0.312676  , 0.        , 0.22860822, ..., 0.28974986,
          1.1120025 , 0.        ]]],


       ...,


       [[[0.57547307, 0.        , 0.        , ..., 0.85850734,
          0.3586921 , 0.        ]]],


       [[[1.0045614 , 0.        , 0.9415052 , ..., 0.        ,
          0.11901689, 0.        ]]],


       [[[0.8553922 , 0.        , 0.2449364 , ..., 0.        ,
          0.02795613, 0.        ]]]], dtype=float32)